De embeddings die hier gescoord worden zijn obv de content, er is niet gekeken naar preprocessing of tokenization

Small part of the data is read in order to rate articles

In [1]:
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output

In [2]:
import pickle
import numpy as np
import pandas as pd
import torch
import tqdm
import random
from csv import writer
from sentence_transformers import SentenceTransformer, util
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [3]:
with open('dataframes\wa_limited_testdf.pickle', 'rb') as handle:
    wa_df = pickle.load(handle)

with open('dataframes\stan_limited_testdf.pickle', 'rb') as handle:
    stan_df = pickle.load(handle)

wa_embeddings = np.load('embeddings\wa_testbatch_embeddings.npy')
stan_embeddings = np.load('embeddings\stan_testbatch_embeddings.npy')

In [4]:
wa_content = wa_df.content
stan_content = stan_df.content

## Initial code to create similarity matrix
Not needed because we dumped it in a pickle file

In [18]:
#from sklearn.metrics.pairwise import cosine_similarity
#from sentence_transformers import util
#similarity = []
#for i in range(len(wa_embeddings)):
#    row=[]
#    for j in range(len(stan_embeddings)):
#        row.append(util.pytorch_cos_sim(wa_embeddings[i], stan_embeddings[j]).item())
#    similarity.append(row)

In [19]:
#with open('similaritymatrix_testdf.pickle', 'wb') as handle:
#    pickle.dump(similarity, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
with open('similaritymatrix_testdf.pickle', 'rb') as handle:
    similarity = pickle.load(handle)

In [7]:
def return_top3_sim(similarity=similarity, index=0, wa_content=wa_content, stan_content=stan_content):
    original_article = wa_content[index]
    top_3_idx = np.argsort(similarity[index])[-3:]
    top_3_values = [stan_content[i] for i in top_3_idx]
    return(original_article, top_3_values, top_3_idx)

In [11]:
#ManualScoredSimList = [0] * 7643
#ManualScoredSimList = [ManualScoredSimList] * 544

544 7643


In [8]:
def createManualScoringMatrix(originalMatrix):
    #First create list of zeroes with length=number of stan articles
    matrix = [0]*len(originalMatrix[0])
    #then multiply this list with amount wa articles
    matrix = matrix * len(originalMatrix)
    return matrix

In [6]:
with open('ManualScoringMatrix.pickle', 'rb') as handle:
    ManualScoredSimList = pickle.load(handle)

with open('IndexesToRate.pickle', 'rb') as handle:
    IndexesToRate = pickle.load(handle)

In [8]:
for i in range(0, len(ManualScoredSimList)):
    for j in range(0,len(ManualScoredSimList[0])):
        ManualScoredSimList[i][j] = int(ManualScoredSimList[i][j])

In [36]:
len(stan_content)

7643

In [38]:
ScoredArticles = np.empty((544, 7643))

In [53]:
IndexesToRate = list(range(544))


In [788]:
len(IndexesToRate)

0

In [13]:
def rate_articles(similarity=similarity, wa_content=wa_content, stan_content=stan_content, ManualScoredSimList=ManualScoredSimList, IndexesToRate=IndexesToRate):
    index = random.choice(IndexesToRate)
    original_article = wa_content[index]
    top_3_idx = np.argsort(similarity[index])[-3:]
    top_3_values = [stan_content[i] for i in top_3_idx]
    UserScores = []
    for article in top_3_idx:
        print(f"Original_article: {original_article} \n \n Similar article: {stan_content[article]}")
        UserScore = int(input("From 1 to 5, how similar are these articles? :"))
        UserScores.append(UserScore)
        ManualScoredSimList[index][article] = UserScore
        clear_output()
    with open('ManualScoringLog.csv', 'a', newline='') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow([index, top_3_idx, UserScores])
        f_object.close()
    IndexesToRate.remove(index)
    return("Scores have been saved in the logfile!")

In [789]:
rate_articles()

IndexError: Cannot choose from an empty sequence

In [781]:
wa_content[543]

"'Buren' met Harold 'Neighbours' of 'Buren' is een heel bekende tv-reeks uit Australië. Ook in Vlaanderen heeft ze veel fans. Die fans zullen binnenkort opkijken. In 2015 bestaat 'Buren' 30 jaar. Dat wordt gevierd. Zo komen de acteurs Ann Charleston en Ian Smith terug. Zij speelden jarenlang het koppel Madge en Harold. Harold was de saaie man met de bril. Smith was er voor het eerst bij in 1987. Zes jaar geleden stopte hij voorgoed in de reeks. Een echte ster werd Smith nooit. Andere acteurs uit 'Buren' lukte dat wel: Kylie Minogue, Jason Donovan, Russell Crowe, Guy Pearce, Natalie Imbruglia\x85  "

In [785]:
ManualScoredSimList[543][4657] = 1

In [784]:
stan_content[4657]

"Nog een Belg naar Hollywood Regisseur Fabrice du Welz gaat ' Message from the king ' draaien met de rijzende ster Chadwick Boseman , ook bekend als ' Black Panther '. Van onze medewerker Jeroen Struys ' Het Belgische label is hot in Hollywood ', vertelt onze landgenoot Fabrice du Welz aan de telefoon vanuit Santa Monica , Californië . Op de American Film Market raakte bekend dat hij na drie Belgische films en één Franse nu een film in Hollywood mag draaien . Message from the king gaat over een man die naar L.A. reist om zijn verdwenen zus te zoeken . Een detectiveverhaal , terwijl Du Welz bekendstaat om zijn behoorlijk edgy horrorfilms - zoals Alleluia , vanaf volgende week bij ons in de cinema . ' Alleluia kreeg veel aandacht in Cannes . De Amerikanen appreciëren de emotionele inzet en de acteerprestaties . Ik wil het gewelddadige contrasteren met fragiliteit . Ook voor deze film wil ik echt in de diepte gaan , en dat was ook wat Chadwick Boseman zo aantrok om de hoofdrol te vertolke

### After rating articles, run next code cell to save

In [655]:
with open('ManualScoringMatrix.pickle', 'wb') as handle:
    pickle.dump(ManualScoredSimList, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('IndexesToRate.pickle', 'wb') as handle:
     pickle.dump(IndexesToRate, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [608]:
testdf = pd.read_csv('ManualScoringLog.csv')
testdf

,Wablieft_idx,top_3_standaard_idx,Manual_Scores
0,449,[7077 5107 4620],"[1, 1, 2]"
1,420,[ 783 2988 4617],"[2, 1, 3]"
2,154,[3651 7595 442],"[2, 2, 4]"
3,333,[2534 2613 5180],"[4, 5, 3]"
4,257,[ 411 3656 1963],"[2, 1, 1]"
...,...,...,...
314,160,[4333 5116 4528],"[3, 3, 3]"
315,170,[7370 3950 1539],"[2, 2, 0]"
316,253,[ 774 1202 2216],"[0, 0, 1]"
317,290,[ 633 1565 2534],"[1, 4, 2]"
